In [1]:
using Pkg

Pkg.activate(".")

  Activating project at `~/Documents/Uni/Master/MA/visual_analysis`


In [11]:
include("utils.jl")
include("visual_utils.jl")

show_eof_modes_of_timeline (generic function with 1 method)

In [3]:
using GLMakie
using GeoMakie
using EmpiricalOrthogonalFunctions
using NCDatasets 
using Dates
using BenchmarkTools
using Statistics

In [35]:
function compare_truth_with_tldata(
    truth::TimelineData,
    data::TimelineData,
    filename::String;
    framerate::Int = 30,
    colormap = :viridis,
    coastline_color = :white,
    colorrange::Union{Nothing, Tuple{<:Real, <:Real}} = nothing, 
    shading = Makie.automatic,
    resolution::Union{Nothing, Tuple{Int, Int}} = nothing
    )::Makie.Figure

    fig = isnothing(resolution) ?  Figure(fontsize=12) : Figure(size = resolution, fontsize=12)

    time_index = Observable(1)


    axis = Dict()

    lon_bounds = extrema(data.lons)
    lat_bounds = extrema(data.lats)

    all_extrema = extrema.([[d.data for d in data.datasets]; [d.data for d in truth.datasets]])

    min_val, max_val = reduce((a, b) -> (min(a[1], b[1]), max(a[2], b[2])), all_extrema)

    axis["truth"] = local_geoaxis_creation!(fig, lon_bounds, lat_bounds; title=@lift("Reference ssp126 $(Dates.format(truth.time[$time_index], "mm/yyyy"))"))

    for (i, dataset) in enumerate(data.datasets)
        axis[dataset.name] = local_geoaxis_creation!(fig, lon_bounds, lat_bounds; title=@lift("$(dataset.name) $(Dates.format(data.time[$time_index], "mm/yyyy"))"), figure_col=i+1)
    end

    Colorbar(fig[2, 1:length(axis)], limits = (min_val, max_val), colormap = colormap, vertical = false,  label = "IVT (norm) in kg s^-1 m^-1")

    truth_slice = @lift(truth.datasets[1].data[:, :, $time_index])

    surface!(axis["truth"], lon_bounds[1]..lon_bounds[2], lat_bounds[1]..lat_bounds[2], truth_slice; shading = shading, colormap = colormap, colorrange = (min_val, max_val))
    lines!(axis["truth"], GeoMakie.coastlines(); color = coastline_color, transformation = (; translation = (0, 0, 1000)))
    
    for dataset in data.datasets

        array_slice = @lift(dataset.data[:, :, $time_index])
        
        surface!(axis[dataset.name], lon_bounds[1]..lon_bounds[2], lat_bounds[1]..lat_bounds[2], array_slice; shading = shading, colormap = colormap, colorrange = (min_val, max_val))
        lines!(axis[dataset.name], GeoMakie.coastlines(); color = coastline_color, transformation = (; translation = (0, 0, 1000)))
    end

    timestamps = eachindex(truth.time)
    record(fig, filename, timestamps; framerate = framerate) do t
        time_index[] = t
    end

    
    return fig
end

compare_truth_with_tldata (generic function with 1 method)

In [5]:
base_path = "/home/denis/workspace/data/ivt_monthly_mean"

"/home/denis/workspace/data/ivt_monthly_mean"

In [13]:
tl_data = filter_by_date(build_timeline_data(base_path, 10, "ssp126", "ssp585"; file_range_selection=4:5)) do timedata
    return year(timedata) >= 2081 && year(timedata) <= 2100  
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2081-01-16T09:00:00"), DateTime("2081-02-14T21:00:00"), DateTime("2081-03-16T09:00:00"), DateTime("2081-04-15T21:00:00"), DateTime("2081-05-16T09:00:00"), DateTime("2081-06-15T21:00:00"), DateTime("2081-07-16T09:00:00"), DateTime("2081-08-16T09:00:00"), DateTime("2081-09-15T21:00:00"), DateTime("2081-10-16T09:00:00")  …  DateTime("2100-03-16T09:00:

In [15]:
truth_data = filter_by_date(build_timeline_data(base_path, 10, "ssp126", "ssp585"; file_range_selection=1:2)) do timedata
    return year(timedata) >= 2015 && year(timedata) <= 2034
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-16T12:00:00"), DateTime("2015-02-14T21:00:00"), DateTime("2015-03-16T09:00:00"), DateTime("2015-04-15T21:00:00"), DateTime("2015-05-16T09:00:00"), DateTime("2015-06-15T21:00:00"), DateTime("2015-07-16T09:00:00"), DateTime("2015-08-16T09:00:00"), DateTime("2015-09-15T21:00:00"), DateTime("2015-10-16T09:00:00")  …  DateTime("2034-03-16T09:00:

In [36]:
compare_truth_with_tldata(truth_data, tl_data, "mon_mean_comparison.mp4"; framerate = 3, resolution = (2000, 1000), colormap = :managua)